In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Ciphense task1

In [ ]:
!pip install flask-ngrok
!pip install geopandas

In [ ]:
import base64
from io import BytesIO
from matplotlib.figure import Figure
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('data_with_loc.csv', header=0)

all_data = pd.DataFrame(columns=['precious_metal','gemstone', 'artificial', 'misc','X', 'Y'])
all_data['precious_metal'] = data['precious_metal']
all_data['gemstone'] = data['gemstone']
all_data['artificial'] = data['artificial']
all_data['misc']=data['misc']
all_data['X'] = data['Y']
all_data['Y'] = data['X']
all_data['X'] = all_data['X'].apply(lambda c: c if (c<=98 and c>=67) else None)
all_data['Y'] = all_data['Y'].apply(lambda v: v if (v<=38 and v>= 5) else None)

import geopandas as gpd                 ###map analysis
import descartes
from shapely.geometry import Polygon, Point
map = gpd.read_file('maps/IND_adm0.shp')
geometry = [Point(xy) for xy in zip(all_data['X'], all_data['Y'])]     
crs = {'init':'epsg:4326'}
geo_df = gpd.GeoDataFrame(all_data, crs = crs, geometry = geometry)
elements = np.array([all_data['precious_metal'].sum(), all_data['gemstone'].sum(), all_data['artificial'].sum(), all_data['misc'].sum()])
labels = ['Metal', 'Gems', 'Artificial', 'Misc.']
colours = ['gold', 'crimson', 'teal', 'darkgreen']

volume = [25.38, 30.67, 45.59, 46.85, 43.63, 40.24, 40.03, 39.29, 43.2, 41.54, 40.19, 35.89]
year = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

@app.route("/")
def index(): 
    return render_template('index.html')


@app.route("/result", methods=['GET', 'POST'])
def ans():
    metal = request.form.getlist("metal")
    gem = request.form.getlist("gem")
    arti = request.form.getlist("arti")
    misc = request.form.getlist("misc")
    print(metal, gem, arti, misc)
    # Generate the figure **without using pyplot**.
    fig, ax = plt.subplots(figsize = (15, 15))
    map.plot( ax = ax, alpha = 0.2, color = 'grey')
    if metal: geo_df[geo_df['precious_metal']>0].plot( ax = ax, markersize= 20, color = 'gold', marker = '.', label = 'Metal', alpha= 0.5)
    if gem: geo_df[geo_df['gemstone']>0].plot( ax = ax, markersize= 10, color = 'crimson', marker = '^', label = 'Gems', alpha = 0.5)
    if arti: geo_df[geo_df['artificial']>0].plot( ax = ax, markersize= 10, color = 'teal', marker = 'o', label = 'Artificial', alpha = 0.5)
    if misc: geo_df[geo_df['misc']>0].plot( ax = ax, markersize= 10, color = 'darkgreen', marker = '*', label = 'Misc.', alpha= 0.5)
    plt.title('Distribution of Jewellery Manufacturers in India')
    plt.legend()

    # Save it to a temporary buffer.
    buf = BytesIO()
    fig.savefig(buf, format="png")
    # Embed the result in the html output.
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    plt.pie(elements, labels= labels, colors= colours, autopct= '%1.0f%%')
    plt.title('Products')
    buf = BytesIO()
    plt.savefig(buf, format="png")
    pie = base64.b64encode(buf.getbuffer()).decode("ascii")
    buf.flush()
    plt.clf()
    plt.bar(year, volume)
    plt.title('Value Exported')
    plt.xlabel('Financial Year')
    plt.ylabel('Export value (bn USD)')
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.show()
    line = base64.b64encode(buf.getbuffer()).decode("ascii")
    return f" <title> Plot </title> <img src='data:image/png;base64,{data}'/> <img src='data:image/png;base64,{pie}'/><img src='data:image/png;base64,{line}'/>"

app.run()